In [17]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..','..','..')))
from pudl import pudl, ferc1, eia923, settings, constants
from pudl import models, models_ferc1, models_eia923
from pudl import clean_eia923, clean_ferc1, clean_pudl
from pudl import analysis
import numpy as np
import pandas as pd
import sqlalchemy as sa
import matplotlib.pyplot as plt
import matplotlib as mpl
pudl_engine = pudl.connect_db()
f1_engine = ferc1.connect_db()
plt.style.use('ggplot')
mpl.rcParams['figure.figsize'] = (10,6)
mpl.rcParams['figure.dpi'] = 150

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Make sure we get an exhaustive list.
#ferc1.init_db(refyear=2016, years=range(2004,2017), def_db=True)

Defining new FERC Form 1 DB based on 2016...
Clearing any existing FERC Form 1 database MetaData...
Ingesting FERC Form 1 Data from 2004...
Ingesting FERC Form 1 Data from 2005...
Ingesting FERC Form 1 Data from 2006...
Ingesting FERC Form 1 Data from 2007...
Ingesting FERC Form 1 Data from 2008...
Ingesting FERC Form 1 Data from 2009...
Ingesting FERC Form 1 Data from 2010...
Ingesting FERC Form 1 Data from 2011...
Ingesting FERC Form 1 Data from 2012...
Ingesting FERC Form 1 Data from 2013...
Ingesting FERC Form 1 Data from 2014...
Ingesting FERC Form 1 Data from 2015...
Ingesting FERC Form 1 Data from 2016...


In [3]:
new_fercplants = analysis.fercplants(new=True, years=[2016,], min_capacity=0.0)
pudl_fercutils = pd.read_sql('''select respondent_id, respondent_name from utilities_ferc;''', pudl_engine)
new_fercutils = new_fercplants[['respondent_id','respondent_name']].drop_duplicates()
new_fercutils = new_fercutils.set_index(['respondent_id',])
pudl_fercutils = pudl_fercutils.set_index(['respondent_id',])
new_index = new_fercutils.index.difference(pudl_fercutils.index)
really_new_fercutils = new_fercutils.loc[new_index].reset_index()
really_new_fercutils

Defining new FERC Form 1 DB based on 2015...
Clearing any existing FERC Form 1 database MetaData...


,respondent_id,respondent_name
0,513,"4C Acquisition, Llc"


In [18]:
all_ferc_respondents = pd.read_sql('''select distinct respondent_id, respondent_name from f1_respondent_id order by respondent_id asc;''', f1_engine)

In [20]:
all_ferc_respondents.to_excel('all_ferc_respondents_2004-2016.xlsx', index=False)

In [4]:
small_plants_2016 = pd.read_sql('''SELECT report_year, respondent_id, row_number, spplmnt_num, plant_name, kind_of_fuel \
                                        FROM f1_gnrt_plant \
                                        WHERE report_year=2016;''', f1_engine)
small_plants_2016['record_number'] = small_plants_2016['row_number'] + 46*small_plants_2016['spplmnt_num']
small_plants_2016 = small_plants_2016.drop(['row_number','spplmnt_num'],axis=1)
small_plants_2016 = small_plants_2016[['report_year','respondent_id','record_number','plant_name','kind_of_fuel']]
small_plants_2016.to_excel('small_plants_2016.xlsx', index=False)

In [5]:
len(new_fercplants)

130

In [6]:
new_fercplants.to_excel('ferc1_unmapped_plants_2016.xlsx')

In [16]:
x = pudl_fercutils.reset_index()

,respondent_id,respondent_name
113,454,"Entergy Louisiana, LLC"
